In [2]:
import numpy as np
import imageio
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA3_256
from Crypto.Signature import pkcs1_15

def image_histogram_normalization(image):
    image_max = np.max(image)
    image_min = np.min(image)
    if image_max - image_min != 0:
        image = ((((image - image_min) / (image_max - image_min)) * (2**8 - 1)).round()).astype(np.uint8)
    return image

def load_public_key(file_path):
    with open(file_path, 'rb') as f:
        public_key = RSA.import_key(f.read())
    return public_key

def compute_image_hash(image_path):
    image = image_histogram_normalization(imageio.imread(image_path))
    tabBits = image.ravel()
    tabBits = tabBits[tabBits != 0]  #Usun zera z wejscia

    processNumbers = []
    tc = 0
    for x in tabBits:
        tc = tc + 1
        if tc < 4000:
            processNumbers.append(x)

    #Zamiana na bajty
    random_bytes = bytes(processNumbers)

    #Generowanie hasza obrazu
    hash_obj = SHA3_256.new(random_bytes)
    return hash_obj

def verify_file(image_path, public_key_path, signature_path):
    #Wczytaj klucz publiczny
    public_key = load_public_key(public_key_path)

    #Obliczanie hasza 
    image_hash = compute_image_hash(image_path)

    #Wczytaj podpis z pliku
    with open(signature_path, 'rb') as f:
        signature = f.read()

    #Porownaj hasz i podpis
    try:
        pkcs1_15.new(public_key).verify(image_hash, signature)
        print("Zdjecie zgodne z oryginalem")
    except (ValueError, TypeError):
        print("Zdjecie niezgodne z oryginalem")

verify_file('psycho_kopia.png', 'public_key.pem', 'image_signature.bin')

Zdjecie zgodne z oryginalem
